<a href="https://colab.research.google.com/github/clarara/wiset/blob/master/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resnet50

In [ ]:
# -*- coding: utf-8 -*-
"""keras-video-classification.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1C2vED82YhLuq8DvtLIavnqAVGGbp3tEI
### 모델 학습 : ResNet50
"""

# set the backend to make output file of model training
import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# from tensorflow.keras.layers import AveragePooling2D
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.layers import Flatten
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Input

from sklearn.preprocessing import LabelBinarizer #one-hot enocding
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

# Commented out IPython magic to ensure Python compatibility.
# %cd /content/drive/My Drive/UNI_project/keras-video-classification
cur_dir=os.getcwd()
#print(cur_dir)

!ls Sports-Type-Classifier/data | grep -Ev "urls|models|csv|pkl" 

#!git clone https://github.com/Choa-Lee/UNI_project.git

!apt install tree

!tree --dirsfirst --filelimit 50

!ls Sports-Type-Classifier/data | grep -Ev "urls|models|csv|pkl" 
#!tree --dirsfirst --filelimit 50

args = { "dataset" : "/content/drive/My Drive/UNI_project/keras-video-classification/Sports-Type-Classifier/data",
        "model" : "/content/drive/My Drive/UNI_project/keras-video-classification/model/activity.model",
            "label_bin" : "/content/drive/My Drive/UNI_project/keras-video-classification/model/lb.pickle" ,
        "epochs" : 50,
        "plot" : "plot2.png"}

print(args)

# initialize the set of labels from the dataset
LABELS = set(["weight_lifting", "tennis", "football"])

# grab the list of images in our dataset directory, then initialize the list of data and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
print(imagePaths)

data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
   # extract the class label from the filename
   label = imagePath.split(os.path.sep)[-2]

   # if the label of the current image is not part of of the labels 
  # which are interested in, then ignore the image
   if label not in LABELS:
      continue

   # load the image, convert it to RGB channel ordering, and resize it 
  # to be a fixed 224x224 pixels, ignoring aspect ratio
   image = cv2.imread(imagePath)
   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
   image = cv2.resize(image, (224, 224))
  # -----------------------------------------------------
  #image = cv2.resize(image, (60, 80))
  # -----------------------------------------------------


  # update the data and labels lists, respectively
   data.append(image)
   labels.append(label)

#print(data)
#print(label)
print(image.shape)

# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels[:3])

# imagePath = '/content/drive/My Drive/UNI_project/keras-video-classification/Sports-Type-Classifier/data/football/00000634.jpg'
# label = imagePath.split(os.path.sep)[-2]
# print(label)

# image = cv2.imread(imagePath)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# print(image.shape)

# image = cv2.resize(image, (60, 80))
# print(image.shape)

# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, 
                                                  stratify=labels, random_state=42)


# initialize the training data augmentation object
# -----------------------------------------------------
# PREPROCESSING !! >> the parameters should be changed
# -----------------------------------------------------
# for Training
trainAug = ImageDataGenerator(rotation_range=30, 
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest")

# for Validation
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order)
mean = np.array([123.68, 116.779, 103.939], dtype="float32")

# set the mean attribute for trainAug and valAug 
trainAug.mean = mean
valAug.mean = mean

"""### 모델 : ResNet-50"""

# load the ResNet-50 network pre-trained with ImageNet weights while chopping the head of the network off, 
# ensuring the head FC layer sets are left off
baseModel = ResNet50(weights="imagenet", include_top=False, 
                     input_tensor=Input(shape=(224, 224, 3)))
# -----------------------------------------------------------
# baseModel = ResNet50(weights="imagenet", include_top=False, 
#                      input_tensor=Input(shape=(60, 80, 3)))
# -----------------------------------------------------------


# -----------------------------------------------------
# FINE-TUNING !!!
# -----------------------------------------------------
# construct the head of the model that will be placed on top of the base model
# assemble a new 'headModel' and suture it onto the 'baseModel'
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)


# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the 'baseModel' and freeze them so that 
# it will not be trained via backpropagation
# they will *not* be updated during the training process
for layer in baseModel.layers:
   layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"]) 
# Stochastic Gradient Descent (SGD)
# lr: learning rate

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

baseModel.summary()

model.summary()

# train only the head of the network for a few epochs 
# (all other layers in the 'baseModel' are frozen) 
# -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(x=trainAug.flow(trainX, trainY, batch_size=32),
   steps_per_epoch=len(trainX) // 32,
   validation_data=valAug.flow(testX, testY),
   validation_steps=len(testX) // 32,
   epochs=args["epochs"])

# evaluate the network on the testing set
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
   predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

# serialize the model to disk
print("[INFO] serializing network...")
model.save(args["model"], save_format="h5")

# serialize the label binarizer to disk
f = open(args["label_bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

result = plt.imread('./plot2.png')
plt.imshow(result)
plt.show()

"""### 모델 TEST : Video Classification
`Transfer learning`
"""

!ls model

from tensorflow.keras.models import load_model
from collections import deque #for rolling average algorithm
import numpy as np
import argparse
import pickle
import cv2
from google.colab.patches import cv2_imshow

args = { "dataset" : "/content/drive/My Drive/UNI_project/keras-video-classification/Sports-Type-Classifier/data",
        "model" : "/content/drive/My Drive/UNI_project/keras-video-classification/model/activity.model",
            "label_bin" : "/content/drive/My Drive/UNI_project/keras-video-classification/model/lb.pickle" ,
        "input" : "/content/drive/My Drive/UNI_project/keras-video-classification/example_clips/lifting.mp4",
        "output" : "/content/drive/My Drive/UNI_project/keras-video-classification/output/lifting_128.avi",
        "size" : 128}

# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(args["model"])
lb = pickle.loads(open(args["label_bin"], "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=args["size"])

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(args["input"])
writer = None
(W, H) = (None, None)

# loop over frames from the video file stream
while True:
   # read the next frame from the file
   (grabbed, frame) = vs.read()

   # if the frame was not grabbed, then we have reached the end
   # of the stream
   if not grabbed:
      break

   # if the frame dimensions are empty, grab them
   if W is None or H is None:
      (H, W) = frame.shape[:2]

   # clone the output frame, then convert it from BGR to RGB
   # ordering, resize the frame to a fixed 224x224, and then
   # perform mean subtraction
   output = frame.copy()
   frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   frame = cv2.resize(frame, (224, 224)).astype("float32")
   frame -= mean
  # -----------------------------------------------------
   # frame = cv2.resize(frame, (60, 80)).astype("float32")
  # -----------------------------------------------------
  
  # Rolling prediction averaging
   # make predictions on the frame and then update the predictions
   # queue
   preds = model.predict(np.expand_dims(frame, axis=0))[0]
   Q.append(preds)

   # perform prediction averaging over the current history of
   # previous predictions
   results = np.array(Q).mean(axis=0)
   i = np.argmax(results)
   label = lb.classes_[i]

   # draw the activity on the output frame
   text = "activity: {}".format(label)
   cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
      1.25, (0, 255, 0), 5)

   # check if the video writer is None
   if writer is None:
      # initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter(args["output"], fourcc, 30,
         (W, H), True)

   # write the output frame to disk
   writer.write(output)

   # show the output image
   # cv2.imshow("Output", output)
   cv2_imshow(output)
   key = cv2.waitKey(1) & 0xFF
  
   # if the `q` key was pressed, break from the loop
   if key == ord("q"):
      break

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()







"""### IR-hand dataset"""

args = { "dataset" : "/content/drive/My Drive/UNI_project/leapGestRecog/",
        "model" : "/content/drive/My Drive/UNI_project/keras-video-classification/model/activity.model",
            "label_bin" : "/content/drive/My Drive/UNI_project/keras-video-classification/model/lb.pickle" ,
        "input" : "/content/drive/My Drive/UNI_project/leapGestRecog/input",
        "output" : "/content/drive/My Drive/UNI_project/leapGestRecog/output/lifting_128.avi",
        "size" : 128}
© 2020 GitHub, Inc.